In [1]:
%cd ../..

E:\システムトレード入門\trade_system_git_workspace


In [2]:
from tqdm.notebook import tqdm

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytz import timezone
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
import pfrl

In [4]:
from scipy.special import softmax

In [5]:
import py_restart

In [6]:
from get_stock_price import StockDatabase

In [7]:
from portfolio.trade_transformer import PortfolioTransformer, PortfolioRestrictorIdentity, FeeCalculatorFree
from portfolio.price_supply import StockDBPriceSupplier

In [8]:
from portfolio.rl_base.envs import TradeEnv, TickerSampler, DatetimeSampler, SamplerManager, PortfolioVectorSampler, ConstSamper
from portfolio.rl_base.basis_func import ComposeFunction, PriceNormalizeConst, MeanCostPriceNormalizeConst, State2Feature

In [9]:
from visualization import visualize_portfolio_rl_bokeh

## グローバルパラメータ―

In [10]:
jst = timezone("Asia/Tokyo")
start_datetime = jst.localize(datetime.datetime(2020,11,10,0,0,0))
end_datetime = jst.localize(datetime.datetime(2020,11,20,0,0,0))
ticker_number = 19
window = np.arange(0,50)
episode_length = 300
freq_str = "5T"

## 環境の作成 

In [11]:
db_path = Path("db/sub_stock_db/nikkei_255_stock_v2.db")

ticker_codes_df = pd.read_csv(Path("portfolio/rl_base/nikkei225_modified_v2.csv"), header=0)  # 自分で作成
ticker_codes = ticker_codes_df["code"].values.astype(str).tolist()

In [12]:
# stock_db
stock_db = StockDatabase(db_path)

# sampler
ticker_names_sampler = TickerSampler(all_ticker_names=ticker_codes,
                                     sampling_ticker_number=ticker_number)


#ticker_names_sampler = ConstSamper(np.random.choice(ticker_codes, ticker_number))  # 固定する



start_datetime_sampler = DatetimeSampler(start_datetime=start_datetime,
                                         end_datetime=end_datetime,
                                         episode_length=episode_length,
                                         freq_str=freq_str
                                        )

portfolio_sampler = PortfolioVectorSampler(ticker_number+1)


sampler_manager = SamplerManager(ticker_names_sampler=ticker_names_sampler,
                                 datetime_sampler=start_datetime_sampler,
                                 portfolio_vector_sampler=portfolio_sampler,
                                )


# PriceSupplierの設定
price_supplier = StockDBPriceSupplier(stock_db,
                                      [],  # 最初は何の銘柄コードも指定しない
                                      episode_length,
                                      freq_str,
                                      interpolate=True
                                     )

# PortfolioTransformerの設定
portfolio_transformer = PortfolioTransformer(price_supplier,
                                             portfolio_restrictor=PortfolioRestrictorIdentity(),
                                             use_ohlc="Close",
                                             initial_all_assets=1e6,  # 学習には関係ない
                                             fee_calculator=FeeCalculatorFree()
                                            )

# TradeEnvの設定
trade_env = TradeEnv(portfolio_transformer,
                     sampler_manager,
                     window=window,
                     fee_const=0.0025
                    )

In [13]:
portfolio_state, _,_,_ = trade_env.reset()
#print(portfolio_state)

In [14]:
portfolio_state.price_array.shape

(20, 50)

In [15]:
portfolio_state.now_price_array.shape

(20,)

## 前処理の設定 

In [16]:
state_transform = ComposeFunction({"price_normalizer":PriceNormalizeConst(None),
                                   "mca_normalizer":MeanCostPriceNormalizeConst(None),
                                   "state2feature":State2Feature()
                                  })

##  モデルの定義 

今回は決定論的な方策を利用する

In [17]:
class DPolicy(nn.Module):
    def __init__(self, in_channels=3, out_number=20):
        super(DPolicy, self).__init__()
        self.out_number = out_number
        
        self.conv1 = nn.Conv2d(in_channels, 12, kernel_size=5, padding=2, stride=2)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(12, self.out_number, kernel_size=5, padding=2, stride=(2,3))
        self.bn2 = nn.BatchNorm2d(self.out_number)
        self.conv3 = nn.Conv2d(self.out_number, self.out_number, kernel_size=(4,3), padding=(2,1), stride=(2,3))
        self.bn3 = nn.BatchNorm2d(self.out_number)

        self.avgpool = nn.AvgPool2d(kernel_size=3)
        self.head = pfrl.policies.DeterministicHead()
        
        
    def forward(self, x):
        #from IPython.core.debugger import Pdb; Pdb().set_trace()
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = self.avgpool(x)
        x = torch.reshape(x, (-1, self.out_number))
        x = F.softmax(x, dim=-1)
        out = self.head(x)
        return out

In [18]:
random_x = torch.randn((30, 3, 20, 50))
policy = DPolicy(3, 20)

In [19]:
out = policy(random_x)
print(out.batch_shape, out.event_shape)
print(out.rsample(torch.Size([])).shape)

torch.Size([30]) torch.Size([20])
torch.Size([30, 20])


In [20]:
class QFunc(nn.Module):
    def __init__(self, in_channels=3, action_size=20):
        super(QFunc, self).__init__()        
        self.conv1 = nn.Conv2d(in_channels, 12, kernel_size=5, padding=2, stride=2)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(12, 24, kernel_size=5, padding=2, stride=(2,3))
        self.bn2 = nn.BatchNorm2d(24)
        self.conv3 = nn.Conv2d(24, 64, kernel_size=(4,3), padding=(2,1), stride=(2,3))
        self.bn3 = nn.BatchNorm2d(64)
        self.avgpool = nn.AvgPool2d(kernel_size=3)

        self.fc1 = nn.Linear(action_size, 64)
        self.fcbn1 = nn.BatchNorm1d(64)

        self.concat = pfrl.nn.ConcatObsAndAction()
        self.fc2 = nn.Linear(64*2, 256)
        self.fcbn2 = nn.BatchNorm1d(256)
        self.head = nn.Linear(256, 1)

    def forward(self, obs_and_action):
        #from IPython.core.debugger import Pdb; Pdb().set_trace()
        obs, action = obs_and_action
        x = F.relu(self.bn1(self.conv1(obs)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.avgpool(x)
        x = torch.reshape(x, (-1, 64))

        y = F.relu(self.fcbn1(self.fc1(action)))

        x = self.concat((x, y))
        x = F.relu(self.fcbn2(self.fc2(x)))
        out = self.head(x)
        return out

In [21]:
random_obs = torch.randn(30, 3, 20, 50)
random_action = torch.randn(30, 20)

In [22]:
qfunc = QFunc(3, 20)
out = qfunc((random_obs, random_action))

In [23]:
out.shape

torch.Size([30, 1])

## エージェントの作成 

In [24]:
policy = DPolicy(in_channels=3, out_number=ticker_number+1)
q_func = QFunc(in_channels=3, action_size=ticker_number+1)

opt_p = torch.optim.Adam(policy.parameters())
opt_q = torch.optim.Adam(q_func.parameters())

rbuf = pfrl.replay_buffers.ReplayBuffer(10 ** 6)

#explorer = pfrl.explorers.AdditiveGaussian(
#    scale=np.array([0.1]*(ticker_number+1)), low=np.array([0]*(ticker_number+1)), high=np.array([1]*(ticker_number+1))
#)

def action_sample():
    portfolio_vector = PortfolioVectorSampler(ticker_number+1).sample()
    return portfolio_vector.astype(np.float32)
    

explorer = pfrl.explorers.ConstantEpsilonGreedy(epsilon=0.3,
                                                random_action_func=action_sample
                                               )

def burnin_action_func():
    """Select random actions until model is updated one or more times."""
    random_x = np.random.uniform(np.array([0]*(ticker_number+1)), np.array([1]*(ticker_number+1)))
    out = softmax(random_x).astype(np.float32)
    return out
    #return np.random.uniform(np.array([0]), np.array([1])).astype(np.float32)
    
gpu = -1

phi = lambda x: x.astype(np.float32, copy=False)

ddpg_agent = pfrl.agents.DDPG(
    policy,
    q_func,
    opt_p,
    opt_q,
    rbuf,
    phi=phi,
    gamma=0.99,
    explorer=explorer,
    replay_start_size=10000,
    target_update_method="soft",
    target_update_interval=1,
    update_interval=1,
    soft_update_tau=5e-3,
    n_times_update=1,
    gpu=gpu,
    minibatch_size=16,
    burnin_action_func=burnin_action_func,
)

## 学習のための関数 

In [25]:
def episode(env,
            agent,
            state_transform,
            return_state_reward=True,
            field_list=["now_price_array", "portfoilo_vector", "mean_cost_price_array", "all_assets", "datetime"],
            seed=None,
            print_span=None,
            is_observe=True):
    
    state_list = []
    reward_list = []
    
    portfolio_state,reward,_,_ = env.reset(seed)
    
    #state_transformの設定
    state_transform.price_normalizer.const_array = portfolio_state.now_price_array
    state_transform.mca_normalizer.const_array = portfolio_state.now_price_array
    
    
    state_list.append(portfolio_state.partial(*field_list))
    reward_list.append(reward)
    

    R = 0
    t = 1
    
    obs = state_transform(portfolio_state)
    
    if print_span is not None:
        print("initial:, all_assets:{}".format(portfolio_state.all_assets))

    while True:
        action = agent.act(obs)
        portfolio_state, reward, done, info = env.step(action)

        state_list.append(portfolio_state.partial(*field_list))
        reward_list.append(reward)

        R += reward
        t += 1
        reset = False
        
        # state前処理
        obs = state_transform(portfolio_state)

        if is_observe:  # 観測(学習)する場合
            agent.observe(obs, reward, done, reset)

        if done:
            break
        if print_span is not None:
            if t%print_span==0:
                print("\tt={}:, all_assets:{}".format(t,portfolio_state.all_assets))
    
    if print_span is not None:
        print("finished(t={}):, all_assets:{}".format(t, portfolio_state.all_assets))
    
    out_dict = {}
    out_dict["R"] = R 
    
    if return_state_reward:
        out_dict["state_list"] = state_list
        out_dict["reward_list"] = reward_list
            
    return out_dict

##  エージェント名

In [26]:
now_datetime = datetime.datetime.now()
now_str = now_datetime.strftime("%Y_%m_%d__%H_%M_%S")
agent_name = now_str
print(agent_name)

2021_05_15__18_52_44


## 一時保存用のオブジェクト

In [27]:
temp_save_dict = {"agent_name":agent_name}

## 一時保存用の設定 

In [28]:
temp_filepath = Path("portfolio/rl_base/training_temp.tmp")

object_temp_filepath = Path("portfolio/rl_base/temp_save_dict.pickle")

save_funcs = [ddpg_agent.save]
load_funcs = [ddpg_agent.load]
func_paths = [Path("portfolio/rl_base/temp_agent")]

## オブジェクトの登録とロード

In [29]:
counter = py_restart.enable_counter(temp_filepath, each_save=True, save_span=100, use_tempfile=True)

temp_save_dict = counter.save_load_object(temp_save_dict, object_temp_filepath)

counter.save_load_funcs(save_funcs=save_funcs,
                        load_funcs=load_funcs,
                        func_paths=func_paths
                        )

## 画像を保存するディレクトリ 

In [30]:
save_fig_dir_path = Path("portfolio/rl_base/trading_process_figures") / Path(agent_name)
if not save_fig_dir_path.exists():
    save_fig_dir_path.mkdir()

## 学習のパラメータ― 

In [31]:
n_episodes = 1000

## 学習のイテレーション 

In [32]:
with counter:
    for i in counter(tqdm(range(1, n_episodes + 1))):
        out_dict = episode(trade_env,
                           ddpg_agent,
                           state_transform,
                          )        

        if i%50 == 0:
            print("episode:{}, return:{}".format(i, out_dict["R"]))
        if i%100 == 0:
            print("statistics:", ddpg_agent.get_statistics())

        if i%50 == 0:
            with ddpg_agent.eval_mode():
                out_dict = episode(trade_env, 
                                   ddpg_agent,
                                   state_transform,
                                   return_state_reward=True,
                                   field_list=["names", "now_price_array", "portfolio_vector", "mean_cost_price_array", "all_assets", "datetime"],
                                   is_observe=False
                                   )
                
                save_fig_path = save_fig_dir_path / Path("trading_process_i_{}.png".format(i))
                visualize_portfolio_rl_bokeh(out_dict["state_list"], out_dict["reward_list"], save_path=save_fig_path, is_show=False, is_save=True, is_jupyter=True)
            
print("Finshed")

episode:50, return:0.8524706200300625
episode:100, return:-0.8709747566855807
statistics: [('average_q', -229.31631), ('average_actor_loss', 229.78707382202148), ('average_critic_loss', 276390.81959869387), ('n_updates', 20001)]



KeyboardInterrupt: 

## 学習結果の可視化

In [ ]:
with ddpg_agent.eval_mode():
    out_dict = episode(trade_env, 
                       ddpg_agent,
                       state_transform,
                       return_state_reward=True,
                       field_list=["names", "now_price_array", "portfolio_vector", "mean_cost_price_array", "all_assets", "datetime"],
                       is_observe=False
                       )
    
visualize_portfolio_rl_bokeh(out_dict["state_list"], out_dict["reward_list"], is_show=False, is_jupyter=True)

##  エージェントの保存

In [ ]:
save_agent_path = Path("portfolio/rl_base/saved_agents") / Path(agent_name)
ddpg_agent.save(save_agent_path)